In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('HR_DATA.csv', delimiter=';')

df.head()

,RequestID,EmpID,EmpName,Dept,Request_Date,Completion_Date,Request_Detail,PIC
0,REQ0001,103,Charlie Davis,HR,01/01/2024,09/01/2024,Payroll Issue,Sarah Supervisor
1,REQ0002,103,Charlie Davis,HR,01/01/2024,02/01/2024,Access Approval,John Manager
2,REQ0003,105,Eve Wilson,HR,01/01/2024,03/01/2024,Access Approval,John Manager
3,REQ0004,104,David Brown,HR,02/01/2024,10/01/2024,Payroll Issue,Sarah Supervisor
4,REQ0005,102,Bob Smith,HR,02/01/2024,04/01/2024,Benefit Claim,John Manager


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   RequestID        1000 non-null   object
 1   EmpID            1000 non-null   int64 
 2   EmpName          1000 non-null   object
 3   Dept             1000 non-null   object
 4   Request_Date     991 non-null    object
 5   Completion_Date  907 non-null    object
 6   Request_Detail   994 non-null    object
 7   PIC              995 non-null    object
dtypes: int64(1), object(7)
memory usage: 62.6+ KB


In [6]:
df.isna().sum()

RequestID           0
EmpID               0
EmpName             0
Dept                0
Request_Date        9
Completion_Date    93
Request_Detail      6
PIC                 5
dtype: int64

**Filling Missing Values**

In [ ]:
df['Request_Date'] = pd.to_datetime(df['Request_Date'],format='%d/%m/%Y')
df['Completion_Date'] = pd.to_datetime(df['Completion_Date'],format='%d/%m/%Y', errors='coerce')

df['Request_Date']
df[['Request_Date','Completion_Date']]


,Request_Date,Completion_Date
0,2024-01-01,2024-01-09
1,2024-01-01,2024-01-02
2,2024-01-01,2024-01-03
3,2024-01-02,2024-01-10
4,2024-01-02,2024-01-04
...,...,...
995,NaT,2025-01-09
996,2024-12-30,NaT
997,2024-12-30,2025-01-03
998,2024-12-31,2025-01-07


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   RequestID        1000 non-null   object        
 1   EmpID            1000 non-null   int64         
 2   EmpName          1000 non-null   object        
 3   Dept             1000 non-null   object        
 4   Request_Date     991 non-null    datetime64[ns]
 5   Completion_Date  907 non-null    datetime64[ns]
 6   Request_Detail   994 non-null    object        
 7   PIC              995 non-null    object        
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 62.6+ KB


In [28]:
df['Processing_Days'] = np.busday_count(df['Request_Date'].values.astype('datetime64[D]'), df['Completion_Date'].values.astype('datetime64[D]'))


ValueError: Cannot compute a business day count with a NaT (not-a-time) date